In [1]:
# Computes Atlantic basin - not yet converted to working code!!!!
%load_ext autoreload
%autoreload 2

import jCMIP as jc
import matplotlib.pyplot as plt
import copy
from netCDF4 import Dataset
import numpy as np
import sys

In [2]:
# Atlantic Basin:
def atl_basin(tmask):
    atl_mask = copy.deepcopy(tmask)
    # Southern Ocean:
    atl_mask[np.where((lat < -30) & ((lon > 20) | (lon < -71)))] = 0
    # Arctic:
    atl_mask[np.where((lat > 67) & ((lon > 105) | (lon < -90)))] = 0
    # Remove Black Sea:
    atl_mask[np.where(((lat > 41) & (lat < 45)) & ((lon > 22) & (lon < 45)))] = 0
    
    return atl_mask

# Finds all points in a closed region:
def make_mask(A,sty,stx):
    ni = np.size(A,axis=1)
    nj = np.size(A,axis=0)
    
    if A[sty,stx] == 0:
        print('invalid starting point')
    else:
        A[sty,stx] = 2
        # left
        ptx = stx - 1
        pty = sty
        if ptx == -1:
            ptx = ni-1
        if A[pty,ptx] == 1:
            make_mask(A,pty,ptx)
                
        # up
        ptx = stx
        pty = sty + 1
        if pty < nj:   
            if A[pty,ptx] == 1:
                make_mask(A,pty,ptx)
        
        # right 
        ptx = stx + 1
        pty = sty
        if ptx == ni:
            ptx = 0
        if A[pty,ptx] == 1:
                make_mask(A,pty,ptx)
        
        # down
        ptx = stx
        pty = sty - 1
        if pty >= 0:
            if A[pty,ptx] == 1:
                make_mask(A,pty,ptx) 
                
# Saves Atlantic mask file:
def save_mask(infile,outfile):
    ncid = Dataset(infile,'r')
    dims = ncid.variables['tmask'].get_dims()
    ncid.close
    
    ncid = Dataset(outfile, 'w', format='NETCDF4')
    # dimensions:
    ncid.createDimension(dims[3].name,dims[3].size)
    ncid.createDimension(dims[2].name,dims[2].size)
    ncid.createDimension(dims[1].name,1)
    ncid.createDimension(dims[0].name,None)
    # variables:
    ncid.createVariable('glo_mask','f8',(dims[0].name,dims[1].name,dims[2].name,dims[3].name,))
    ncid.createVariable('atl_mask','f8',(dims[0].name,dims[1].name,dims[2].name,dims[3].name,))
    # fill variables:
    ncid.variables['glo_mask'][0,0,:,:] = glo_mask
    ncid.variables['atl_mask'][0,0,:,:] = atl_mask
    # close:
    ncid.close()

In [ ]:
# Make basin masks:

Model   = 'ACCESS1-0'
datadir = '/home/users/jmecking001/MyCMIP5/'
infile  = (datadir + Model + '/mesh_mask.nc')
outfile = (datadir + Model + '/basins.nc')

ncid  = Dataset(infile,'r')
tmask = ncid.variables['tmask'][0,0,:,:]
lon   = ncid.variables['tlon'][:,:]
lat   = ncid.variables['tlat'][:,:]
ncid.close()

lon[np.where(lon > 180)]  = lon[np.where(lon > 180)]  - 360
lon[np.where(lon < -180)] = lon[np.where(lon < -180)] + 360

# Make Atlantic Basin:
atl_mask = atl_basin(tmask)
# Suez Canal:
# Panama Canal:

stx = 250
sty =  40
sys.setrecursionlimit(np.size(tmask,0)*np.size(tmask,1))
make_mask(atl_mask,sty,stx)
atl_mask[np.where(atl_mask != 2)] = 0
atl_mask = atl_mask/2
# Tidy Arctic Division:
atl_mask[275:,200:300] = tmask[275:,200:300]

# Make Indian Ocean Basin:
ind_mask = ind_basin(tmask,atl_mask)
# Tidy Indian Basin:
ind_mask[110:130,25:30] = tmask[110:130,25:30]
ind_mask[110:125,25:35] = tmask[110:125,25:35]

stx =  20
sty =  40
sys.setrecursionlimit(np.size(tmask,0)*np.size(tmask,1))
make_mask(ind_mask,sty,stx)
ind_mask[np.where(ind_mask != 2)] = 0
ind_mask = ind_mask/2

# Make Pacific Ocean Basin:
pac_mask = pac_basin(tmask,atl_mask,ind_mask)

stx = 150
sty =  40
sys.setrecursionlimit(np.size(tmask,0)*np.size(tmask,1))
make_mask(pac_mask,sty,stx)
pac_mask[np.where(pac_mask != 2)] = 0
pac_mask = pac_mask/2

# Make new Global Basin (i.e. remove internal seas):
glo_mask = atl_mask+ind_mask+pac_mask

# Save to file:
save_mask(infile,outfile)

#
fig = plt.figure(figsize=(25,15))
plt.pcolormesh(tmask+glo_mask+2*pac_mask+3*ind_mask+4*atl_mask)
plt.colorbar()